# Requirements

In [ ]:
from pathlib import Path
kaggle = Path("/content/kaggle.json")

!sudo apt-get install -y git-lfs
!pip install wandb
!pip install kaggle
!pip install sentencepiece
!pip install transformers
!pip install pytorch_lightning
!pip install madgrad
!git clone https://github.com/bloodwass/mixout
!git config --global user.email "simonmeoni@aol.com"
!git config --global user.name "Simon Meoni"
%run /content/mixout/mixout.py
%run /content/mixout/module.py

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c commonlitreadabilityprize
!kaggle datasets download simonmeoni/litbank
!kaggle datasets download simonmeoni/paraphrase-clrp
!unzip train.csv.zip
!unzip paraphrase-clrp.zip

In [ ]:
import pandas as pd
import os
import torch
import gc
import pytorch_lightning as pl
import madgrad
import wandb
import math
import numpy as np
from torch import nn
import torch.nn.functional as F
import concurrent.futures
from transformers import (
    AutoTokenizer,
    AlbertModel,
    AutoModel,
    AdamW,
    AutoModelForMaskedLM,
    LineByLineTextDataset,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer,
    AutoModelForSeq2SeqLM,
)
import transformers
from torchvision import transforms, utils
from pytorch_lightning.loggers import WandbLogger
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.stochastic_weight_avg import StochasticWeightAveraging
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning import Trainer
import glob
import random
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
from torch.optim.lr_scheduler import CosineAnnealingLR

# Paraphrase Dataset

In [ ]:
dataset = pd.read_csv('train.csv')
tokenizer = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")
model = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws").to('cuda')
for index, row in dataset.iterrows():
  sentence = row['excerpt']
  encoding = tokenizer.encode_plus(sentence, return_tensors="pt")
  input_ids, attention_masks = encoding["input_ids"].to('cuda'), encoding["attention_mask"].to('cuda')
  outputs = model.generate(
      input_ids=input_ids, attention_mask=attention_masks,
      max_length=256,
      do_sample=True,
      top_k=120,
      top_p=0.95,
      early_stopping=True,
      num_return_sequences=5
  )
  dataset.at[index, 'excerpt'] = tokenizer.decode(random.choice(outputs), skip_special_tokens=True)
dataset.to_csv('paraphrase.csv')
!mkdir paraphrase-clrp 
!mv paraphrase.csv /content/paraphrase-clrp/paraphrase.csv 
!kaggle datasets metadata -p /content/paraphrase-clrp simonmeoni/paraphrase-clrp